# Beatbot
This project is an AI-powered music generation system that uses ML and DL to generate music when a seed notes are given as input.

In [2]:
import pandas as pd
import tensorflow as tf


ModuleNotFoundError: No module named 'tensorflow'

In [3]:
sudo apt install -y fluidsynth

SyntaxError: invalid syntax (<ipython-input-3-6b4bed7cc9b9>, line 1)

In [ ]:
pip install